# Introduction to RNN

We use Nietsche text data to showcase how Recurrant neural networks work.

## Setup libraries and functions.

We start by importing various libraries needed.

In [1]:
# Plots displayed inline in notebook
%matplotlib inline

# Make Python 3 consistent
from __future__ import print_function, division

# Make help libraries available
import sys

sys.path.append('/home/ubuntu/personal-libraries')

In [149]:
import numpy as np
import pdb
import math

import keras
from keras.models import Model, Sequential
from keras.layers import Embedding, Input, BatchNormalization
from keras.layers.recurrent import SimpleRNN, LSTM
from keras.layers.core import Dense, Flatten
from keras.layers.merge import add
from keras.layers.wrappers import TimeDistributed
from keras.optimizers import Adam
from keras.utils import get_file, to_categorical

from keras import backend as K
K.set_image_data_format('channels_first')

import theano
from theano import shared, tensor as T
from theano.tensor.nnet import nnet

from numpy.random import normal
from itertools import chain
from collections import OrderedDict

## Setup data

We're going to download the collected works of Nietzsche to use as our data

In [3]:
path = get_file('nietzsche.txt',
                origin = "https://s3.amazonaws.com/text-datasets/nietzsche.txt")

text = open(path).read()
print('corpus length:', len(text))

corpus length: 600901


Let's consider an example. We see that the text is very convoluted (haha).

In [4]:
!tail {path} -n25

are thinkers who believe in the saints.


144

It stands to reason that this sketch of the saint, made upon the model
of the whole species, can be confronted with many opposing sketches that
would create a more agreeable impression. There are certain exceptions
among the species who distinguish themselves either by especial
gentleness or especial humanity, and perhaps by the strength of their
own personality. Others are in the highest degree fascinating because
certain of their delusions shed a particular glow over their whole
being, as is the case with the founder of christianity who took himself
for the only begotten son of God and hence felt himself sinless; so that
through his imagination--that should not be too harshly judged since the
whole of antiquity swarmed with sons of god--he attained the same goal,
the sense of complete sinlessness, complete irresponsibility, that can
now be attained by every individual through science.--In the same manner
I have viewed the saints of India

We then make a list of all charcters used in the text. And determine the size of the vocabulary.

In [5]:
chars = sorted(list(set(text)))
vocab_size = len(chars)+1
print('total chars:', vocab_size)

total chars: 86


Sometimes it's useful to have a zero value in the dataset, e.g. for padding, so we add that.

In [6]:
chars.insert(0, "\0")

In [7]:
''.join(chars[1:-6])

'\n !"\'(),-.0123456789:;=?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_abcdefghijklmnopqrstuvwxyz'

We then make a dict for characters and for indices. So each character gets mapped to an indecy and back again.

In [8]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

idx will be the data we use from now own - it simply converts all the characters to their index (based on the mapping above)

In [9]:
idx = [char_indices[c] for c in text]

Consider the first ten characters.

In [10]:
idx[:10]

[40, 42, 29, 30, 25, 27, 29, 1, 1, 1]

Corresponding to (Note the annoying line feeds)

In [11]:
''.join(indices_char[i] for i in idx[:10])

'PREFACE\n\n\n'

## 3 character unrolled model

We make a model to predict the fourth character based on the previous three caracters. We do this using the standard functional API in Keras. So we showcase how to build a RNN like model the hard way.

### Create inputs

Create a list of every 4th character, starting at the 0th, 1st, 2nd, then 3rd characters.

In [12]:
cs = 3
c1_dat = [idx[i] for i in xrange(0, len(idx) - 1 - cs, cs)]
c2_dat = [idx[i + 1] for i in xrange(0, len(idx) - 1 - cs, cs)]
c3_dat = [idx[i + 2] for i in xrange(0, len(idx) - 1 - cs, cs)]
c4_dat = [idx[i + 3] for i in xrange(0, len(idx) - 1 - cs, cs)]

We convert to numpy arrays by stacking, and thus get the inputs to out model.

In [13]:
x1 = np.stack(c1_dat[:-2])
x2 = np.stack(c2_dat[:-2])
x3 = np.stack(c3_dat[:-2])

And the same to get our labels.

In [14]:
y = np.stack(c4_dat[:-2])

So we can consider the first four inputs and outputs.

In [15]:
x1[:4], x2[:4], x3[:4]

(array([40, 30, 29,  1]), array([42, 25,  1, 43]), array([29, 27,  1, 45]))

In [16]:
y[:4]

array([30, 29,  1, 40])

Note the shame. We have 1 dimensional arrays of the same length.

In [17]:
x1.shape, x2.shape, x3.shape, y.shape

((200297,), (200297,), (200297,), (200297,))

We then state the number of latent factors to create in the embedding (i.e. the size of the embedding matrix)

In [18]:
n_fac = 42

Create inputs and embedding outputs for each of our three character inputs. (Note each use the same, so we create it as a function returning the input and output.

In [19]:
def embedding_input(name, n_in, n_out):
    inp = Input(shape = (1, ), dtype = 'int64', name = name)
    emb = Embedding(n_in, n_out, input_length = 1)(inp)
    return inp, Flatten()(emb)

Create input and output for each three character input.

In [20]:
c1_in, c1 = embedding_input('c1', vocab_size, n_fac)
c2_in, c2 = embedding_input('c2', vocab_size, n_fac)
c3_in, c3 = embedding_input('c3', vocab_size, n_fac)

### Create and train unrolled model

Pick a size for our hidden statem

In [21]:
n_hidden = 256

This is the layer operation from input to hidden (the 'green arrow' from our diagram). Note that it is not activated yet, we have yet to apply a layer into it.

In [22]:
dense_in = Dense(n_hidden, activation = 'relu')

Our first hidden activation is simply this function applied to the result of the embedding of the first character.

In [23]:
c1_hidden = dense_in(c1)

This is the layer operation from hidden to hidden, (the 'orange arrow' from our diagram).

In [24]:
dense_hidden = Dense(n_hidden, activation = 'tanh')

Our second and third hidden activations sum up the previous hidden state (after applying dense_hidden) to the new input state.

In [25]:
c2_dense = dense_in(c2)
hidden_2 = dense_hidden(c1_hidden)
c2_hidden = add([c2_dense, hidden_2])

In [26]:
c3_dense = dense_in(c3)
hidden_3 = dense_hidden(c2_hidden)
c3_hidden = add([c3_dense, hidden_3])

This is the layer operation from hidden to output, (the 'blue arrow' from our diagram).

In [27]:
dense_out = Dense(vocab_size, activation = 'softmax')

The third hidden state is the input to our output layer.

In [28]:
c4_out = dense_out(c3_hidden)

We can now define our model using the three inputs (which we defined above), and the output from the final layer.

In [29]:
model = Model([c1_in, c2_in, c3_in], c4_out)

And compile it.

In [30]:
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = Adam())

And finally we can fit the model

In [31]:
model.fit([x1, x2, x3], y, batch_size = 64, epochs = 10)

Epoch 1/10
200297/200297 [==============================] - 12s - loss: 2.4025    
Epoch 2/10
200297/200297 [==============================] - 12s - loss: 2.2607    
Epoch 3/10
200297/200297 [==============================] - 12s - loss: 2.2072    
Epoch 4/10
200297/200297 [==============================] - 12s - loss: 2.1623    
Epoch 5/10
200297/200297 [==============================] - 12s - loss: 2.1292    
Epoch 6/10
200297/200297 [==============================] - 12s - loss: 2.1087    
Epoch 7/10
200297/200297 [==============================] - 12s - loss: 2.0943    
Epoch 8/10
200297/200297 [==============================] - 12s - loss: 2.0847    
Epoch 9/10
200297/200297 [==============================] - 12s - loss: 2.0768    
Epoch 10/10
200297/200297 [==============================] - 12s - loss: 2.0698    


### Test model

We define a function that returns the predictions for a given input sequence of characters.

In [32]:
def get_next(inp):
    idxs = [char_indices[c] for c in inp]
    arrs = [np.array(i)[np.newaxis] for i in idxs]
    p = model.predict(arrs)
    i = np.argmax(p)
    return chars[i]

We then try different inputs in order to test the model

In [33]:
get_next('phi')

'l'

In [34]:
get_next(' th')

'e'

In [35]:
get_next(' an')

'd'

Seems to work great!

## First RNN using low level parts of Keras API

We create a sample unrolled RNN the hard way using the functional API of Keras.

### Create input

This is the size of our unrolled RNN.

In [36]:
cs = 8

For each of 0 through 7, create a list of every 8th character with that starting point. These will be the 8 inputs to out model.

In [37]:
c_in_dat = [[idx[i + n] for i in xrange(0, len(idx) - 1 - cs, cs)] for n in range(cs)]

Then create a list of the next character in each of these series. This will be the labels for our model.

In [38]:
c_out_dat = [idx[i + cs] for i in xrange(0, len(idx) - 1 - cs, cs)]

We can then create the input as a numpy array.

In [39]:
xs = [np.stack(c[:-2]) for c in c_in_dat]

So we have an array of 8.

In [40]:
len(xs), xs[0].shape

(8, (75110,))

And for the labels

In [41]:
y = np.stack(c_out_dat[:-2])

Which is a simple one dimensional array.

In [42]:
y.shape

(75110,)

So each column below is one series of 8 characters from the text.

In [43]:
[xs[n][:cs] for n in range(cs)]

[array([40,  1, 33,  2, 72, 67, 73,  2]),
 array([42,  1, 38, 44,  2,  9, 61, 73]),
 array([29, 43, 31, 71, 54,  9, 58, 61]),
 array([30, 45,  2, 74,  2, 76, 67, 58]),
 array([25, 40, 73, 73, 76, 61, 24, 71]),
 array([27, 40, 61, 61, 68, 54,  2, 58]),
 array([29, 39, 54,  2, 66, 73, 33,  2]),
 array([ 1, 43, 73, 62, 54,  2, 72, 67])]

...and this is the next character after each sequence. Which as we can see is also the first character of each sequence after the first.

In [44]:
y[:cs]

array([ 1, 33,  2, 72, 67, 73,  2, 68])

We then state the number of latent factors to create in the embedding (i.e. the size of the embedding matrix)

In [45]:
n_fac = 42

### Create and train model

We start by defining the embedding input layers again.

In [46]:
def embedding_input(name, n_in, n_out):
    inp = Input(shape=(1, ), dtype = 'int64', name = name + '_in')
    emb = Embedding(n_in, n_out, input_length = 1, name = name + '_emb')(inp)
    return inp, Flatten()(emb)

We then create the embeddings for each eight characters.

In [47]:
c_ins = [embedding_input('c' + str(n), vocab_size, n_fac) for n in range(cs)]

We can then define our dense layers. That is the input, hidden and output dense layers. Note again that these are not activated before we pass something to them.
Also note that we initialise the weights using the identity matrix rather than eg. Xavier initialisation. That way we start out by having the hidden state stay the same for each layer of the RNN. That makes intuitive sense, as absent of any new information, we would imagine each layer to be the same. It also makes sense from an empirical point of view, as demonstrated by [this paper](https://arxiv.org/pdf/1504.00941.pdf).

In [48]:
n_hidden = 256

dense_in = Dense(n_hidden, activation = 'relu')
dense_hidden = Dense(n_hidden, activation = 'relu', kernel_initializer = 'identity')
dense_out = Dense(vocab_size, activation = 'softmax')

The first character of each sequence goes through dense_in(), to create our first hidden activations. Note that each element in the list c_ins is a tuple with the input layer and the output flattened embedding layer. We pass the output to the next layer, in accordance to the functional API of Keras.

In [49]:
hidden = dense_in(c_ins[0][1])

Then for each successive layer we combine the output of dense_in() on the next character with the output of dense_hidden() on the current hidden state, to create the new hidden state.

In [50]:
for i in range(1,cs):
    c_dense = dense_in(c_ins[i][1])
    hidden = dense_hidden(hidden)
    hidden = add([c_dense, hidden])

Putting the final hidden state through dense_out() gives us our output.

In [51]:
c_out = dense_out(hidden)

So now we can create our model.

In [52]:
model = Model([c[0] for c in c_ins], c_out)
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = Adam())

And then fit the model

In [53]:
model.fit(xs, y, batch_size = 64, epochs = 12)

Epoch 1/12
75110/75110 [==============================] - 10s - loss: 2.5354    
Epoch 2/12
75110/75110 [==============================] - 10s - loss: 2.2505    
Epoch 3/12
75110/75110 [==============================] - 10s - loss: 2.1474    
Epoch 4/12
75110/75110 [==============================] - 10s - loss: 2.0765    
Epoch 5/12
75110/75110 [==============================] - 10s - loss: 2.0221    
Epoch 6/12
75110/75110 [==============================] - 10s - loss: 1.9759    
Epoch 7/12
75110/75110 [==============================] - 10s - loss: 1.9352    
Epoch 8/12
75110/75110 [==============================] - 10s - loss: 1.8997    
Epoch 9/12
75110/75110 [==============================] - 10s - loss: 1.8678    
Epoch 10/12
75110/75110 [==============================] - 10s - loss: 1.8398    
Epoch 11/12
75110/75110 [==============================] - 10s - loss: 1.8144    
Epoch 12/12
75110/75110 [==============================] - 10s - loss: 1.7923    


### Test model

Again we define a function to predict the next chracter of a sequence.

In [54]:
def get_next(inp):
    idxs = [np.array(char_indices[c])[np.newaxis] for c in inp]
    p = model.predict(idxs)
    return chars[np.argmax(p)]

And we use some examples to test the performance of our model.

In [55]:
get_next('this is ')

'a'

In [56]:
get_next('part of ')

't'

In [57]:
get_next('queens a')

'n'

Seems to work great again!

## First RNN with high level parts of Keras API

We recreate the recurrent neural network using the higher level api of Keras, instead of doing it manually with the functional api.

We define the number of hidden layers, factors, character steps and vocabulary.

In [58]:
n_hidden, n_fac, cs, vocab_size = (256, 42, 8, 86)

Then define the model using the sequential API and the SimpleRNN class.

Again, note that we initialise the weights using the identity matrix rather than eg. Xavier initialisation. That way we start out by having the hidden state stay the same for each layer of the RNN. That makes intuitive sense, as absent of any new information, we would imagine each layer to be the same. It also makes sense from an empirical point of view, as demonstrated by this paper.

In [59]:
model = Sequential([
    Embedding(vocab_size, n_fac, input_length = cs),
    SimpleRNN(n_hidden, activation = 'relu', recurrent_initializer = 'identity'),
    Dense(vocab_size, activation = 'softmax')
])

In [60]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 8, 42)             3612      
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 256)               76544     
_________________________________________________________________
dense_7 (Dense)              (None, 86)                22102     
Total params: 102,258
Trainable params: 102,258
Non-trainable params: 0
_________________________________________________________________


We can then compile the model

In [61]:
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = Adam())

And train it.

In [62]:
model.fit(np.concatenate(xs, axis = 1), y, batch_size = 64, epochs = 12)

Epoch 1/12
75110/75110 [==============================] - 12s - loss: 2.7546    
Epoch 2/12
75110/75110 [==============================] - 12s - loss: 2.2575    
Epoch 3/12
75110/75110 [==============================] - 12s - loss: 2.0555    
Epoch 4/12
75110/75110 [==============================] - 12s - loss: 1.9184    
Epoch 5/12
75110/75110 [==============================] - 12s - loss: 1.8155    
Epoch 6/12
75110/75110 [==============================] - 12s - loss: 1.7310    
Epoch 7/12
75110/75110 [==============================] - 12s - loss: 1.6643    
Epoch 8/12
75110/75110 [==============================] - 12s - loss: 1.6074    
Epoch 9/12
75110/75110 [==============================] - 12s - loss: 1.5622    
Epoch 10/12
75110/75110 [==============================] - 12s - loss: 1.5198    
Epoch 11/12
75110/75110 [==============================] - 12s - loss: 1.4858    
Epoch 12/12
75110/75110 [==============================] - 12s - loss: 1.4532    


We can then test the model. Again, define a function returning the predicted character.

In [63]:
def get_next_keras(inp):
    idxs = [char_indices[c] for c in inp]
    arrs = np.array(idxs)[np.newaxis,:]
    p = model.predict(arrs)[0]
    return chars[np.argmax(p)]

In [64]:
get_next_keras('this is ')

't'

In [65]:
get_next_keras('part of ')

't'

In [66]:
get_next_keras('queens a')

'n'

Works great!

## Returning sequences using low level part of Keras API

We now predict each character to a given sequence length using the previous characters. We move the output generation inside the loop and make it part of the RNN. So everytime we loop through the RNN, we predict the next character. I.e. we predict char 2 using char 1 and predict char 3 using char 1 and two and predict char 4 using char 1, char 2 and char 3.

### Create inputs

To use a sequence model, we can leave our input unchanged - but we have to change our output to a sequence (of course!)
Here, c_out_dat is identical to c_in_dat, but moved across 1 character.

In [67]:
# Previous
#c_in_dat = [[idx[i + n] for i in xrange(0, len(idx) - 1 - cs, cs)]
#            for n in range(cs)]
# Now
c_out_dat = [[idx[i + n] for i in xrange(1, len(idx) - cs, cs)]
             for n in range(cs)]

So we now also have a sequence of outputs.

In [68]:
ys = [np.stack(c[:-2]) for c in c_out_dat]

Reading down each column shows one set of inputs and outputs.

In [69]:
[xs[n][:cs] for n in range(cs)]

[array([[40],
        [ 1],
        [33],
        [ 2],
        [72],
        [67],
        [73],
        [ 2]]), array([[42],
        [ 1],
        [38],
        [44],
        [ 2],
        [ 9],
        [61],
        [73]]), array([[29],
        [43],
        [31],
        [71],
        [54],
        [ 9],
        [58],
        [61]]), array([[30],
        [45],
        [ 2],
        [74],
        [ 2],
        [76],
        [67],
        [58]]), array([[25],
        [40],
        [73],
        [73],
        [76],
        [61],
        [24],
        [71]]), array([[27],
        [40],
        [61],
        [61],
        [68],
        [54],
        [ 2],
        [58]]), array([[29],
        [39],
        [54],
        [ 2],
        [66],
        [73],
        [33],
        [ 2]]), array([[ 1],
        [43],
        [73],
        [62],
        [54],
        [ 2],
        [72],
        [67]])]

In [70]:
[ys[n][:cs] for n in range(cs)]

[array([42,  1, 38, 44,  2,  9, 61, 73]),
 array([29, 43, 31, 71, 54,  9, 58, 61]),
 array([30, 45,  2, 74,  2, 76, 67, 58]),
 array([25, 40, 73, 73, 76, 61, 24, 71]),
 array([27, 40, 61, 61, 68, 54,  2, 58]),
 array([29, 39, 54,  2, 66, 73, 33,  2]),
 array([ 1, 43, 73, 62, 54,  2, 72, 67]),
 array([ 1, 33,  2, 72, 67, 73,  2, 68])]

### Create and train model

We define the input, hidden and output layers again.

In [71]:
dense_in = Dense(n_hidden, activation = 'relu')
dense_hidden = Dense(n_hidden, activation = 'relu', kernel_initializer = 'identity')
dense_out = Dense(vocab_size, activation = 'softmax', name = 'output')

We're going to pass a vector of all zeros as our starting point - below's our input layers for that: So we have moved the first character inside the loop and now just initialise the hidden lyaer with zeros.

In [72]:
inp1 = Input(shape = (n_fac,), name = 'zeros')
hidden = dense_in(inp1)

We can then define the 8 layers. Note that we create a list of each output.

In [73]:
outs = []

for i in range(cs):
    c_dense = dense_in(c_ins[i][1])
    hidden = dense_hidden(hidden)
    hidden = add([c_dense, hidden])
    # every layer now has an output
    outs.append(dense_out(hidden))

We can then define and compile the model. Compared to the previous model, where we just predicted the final character of a sequence, out model specification now has two changes. 1) The out put is a list of 8 outputs, and 2) We add the first zero layer to our list of inputs.

In [74]:
model = Model([inp1] + [c[0] for c in c_ins], outs)
model.compile(loss='sparse_categorical_crossentropy', optimizer = Adam())

We need to pad out input layers, so we do so.

In [75]:
zeros = np.tile(np.zeros(n_fac), (len(xs[0]), 1))
zeros.shape

(75110, 42)

And train the model.

In [76]:
model.fit([zeros] + xs, ys, batch_size = 64, epochs = 12)

Epoch 1/12
75110/75110 [==============================] - 23s - loss: 20.1548 - output_loss_1: 2.7066 - output_loss_2: 2.5653 - output_loss_3: 2.5157 - output_loss_4: 2.4881 - output_loss_5: 2.4855 - output_loss_6: 2.4684 - output_loss_7: 2.4729 - output_loss_8: 2.4522    
Epoch 2/12
75110/75110 [==============================] - 23s - loss: 17.9093 - output_loss_1: 2.5139 - output_loss_2: 2.3524 - output_loss_3: 2.2395 - output_loss_4: 2.1807 - output_loss_5: 2.1685 - output_loss_6: 2.1507 - output_loss_7: 2.1665 - output_loss_8: 2.1371    
Epoch 3/12
75110/75110 [==============================] - 23s - loss: 17.2997 - output_loss_1: 2.4971 - output_loss_2: 2.3299 - output_loss_3: 2.1799 - output_loss_4: 2.0906 - output_loss_5: 2.0668 - output_loss_6: 2.0443 - output_loss_7: 2.0590 - output_loss_8: 2.0320    
Epoch 4/12
75110/75110 [==============================] - 23s - loss: 16.9270 - output_loss_1: 2.4899 - output_loss_2: 2.3197 - output_loss_3: 2.1482 - output_loss_4: 2.0380 - ou

### Test model

Again define a model returning predictions.

In [77]:
def get_nexts(inp):
    idxs = [char_indices[c] for c in inp]
    arrs = [np.array(i)[np.newaxis] for i in idxs]
    p = model.predict([np.zeros(n_fac)[np.newaxis,:]] + arrs)
    print(list(inp))
    return [chars[np.argmax(o)] for o in p]

And run the model on our examples.

In [78]:
get_nexts(' this is')

[' ', 't', 'h', 'i', 's', ' ', 'i', 's']


['t', 'h', 'e', 't', ' ', 'i', 'n', ' ']

In [79]:
get_nexts('part of ')

['p', 'a', 'r', 't', ' ', 'o', 'f', ' ']


['e', 'n', 'e', 'i', 'o', 'f', ' ', 't']

In [80]:
get_nexts('queens a')

['q', 'u', 'e', 'e', 'n', 's', ' ', 'a']


['u', 'e', 's', 't', 'd', ' ', 'o', 'n']

Actually, going forward one character at a time, it seems to work pretty good all things considered!

## Return sequences using high level part of Keras API

We then create a return sequence model using the highest level Keras API.

We reuse the number of hidden layers, factors, character steps and vocabulary size.

In [81]:
n_hidden, n_fac, cs, vocab_size

(256, 42, 8, 86)

To convert our previous keras model into a sequence model, simply add the 'return_sequences = True' parameter, and add TimeDistributed() around our dense layer. This is due to our output having 8x256 dimensions instead of 1x256 in a usual dense layer. This is due to us generating 8 outputs. So TimeDistributed() ensures that we get a dense layer for each of the 8 outputs, which will share the same weight matrix.

In [82]:
model = Sequential([
    Embedding(vocab_size, n_fac, input_length = cs),
    SimpleRNN(n_hidden, return_sequences = True, activation = 'relu',
              recurrent_initializer = 'identity'),
    TimeDistributed(Dense(vocab_size, activation = 'softmax')),
])

In [83]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 8, 42)             3612      
_________________________________________________________________
simple_rnn_2 (SimpleRNN)     (None, 8, 256)            76544     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 8, 86)             22102     
Total params: 102,258
Trainable params: 102,258
Non-trainable params: 0
_________________________________________________________________


We then compile the model

In [84]:
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = Adam())

We then create the model inputs

In [91]:
xs = [np.stack(c[:-2]) for c in c_in_dat]
xs[0].shape

(75110,)

Redefine ys, as we added a dimension earlier.

In [93]:
ys = [np.stack(c[:-2]) for c in c_out_dat]
ys[0].shape

(75110,)

In [94]:
x_rnn = np.stack(xs, axis = 1)
y_rnn = np.expand_dims(np.stack(ys, axis = 1), -1)

Yielding the expected shape of ((75110, 8), (75110, 8, 1))

In [95]:
x_rnn.shape, y_rnn.shape

((75110, 8), (75110, 8, 1))

We can then train the model and see that we obtain similar loss as earlier.

In [96]:
model.fit(x_rnn, y_rnn, batch_size = 64, epochs = 8)

Epoch 1/8
75110/75110 [==============================] - 15s - loss: 2.4279    
Epoch 2/8
75110/75110 [==============================] - 15s - loss: 2.0019    
Epoch 3/8
75110/75110 [==============================] - 15s - loss: 1.8848    
Epoch 4/8
75110/75110 [==============================] - 15s - loss: 1.8230    
Epoch 5/8
75110/75110 [==============================] - 15s - loss: 1.7847    
Epoch 6/8
75110/75110 [==============================] - 15s - loss: 1.7582    
Epoch 7/8
75110/75110 [==============================] - 15s - loss: 1.7380    
Epoch 8/8
75110/75110 [==============================] - 15s - loss: 1.7225    


And we can then test by defining the test function

In [97]:
def get_nexts_keras(inp):
    idxs = [char_indices[c] for c in inp]
    arr = np.array(idxs)[np.newaxis,:]
    p = model.predict(arr)[0]
    print(list(inp))
    return [chars[np.argmax(o)] for o in p]

In [98]:
get_nexts_keras(' this is')

[' ', 't', 'h', 'i', 's', ' ', 'i', 's']


['t', 'h', 'e', 's', ' ', 's', 's', ' ']

In [99]:
get_nexts_keras('part of ')

['p', 'a', 'r', 't', ' ', 'o', 'f', ' ']


['o', 'r', 't', 'i', 'o', 'f', ' ', 't']

In [100]:
get_nexts_keras('queens a')

['q', 'u', 'e', 'e', 'n', 's', ' ', 'a']


['u', 'i', 's', 's', 't', ' ', 'o', 'n']

In [ ]:
Again a pretty good result

## Stateful model in Keras

How do we add state to a RNN? How to generate a model, that can handle long term dependencies?

We alter the previous model by 1) not shuffling the input data and 2) stopping passing in zeroes for each itteration. That is we want to keep the hidden state for each itteration of 8 characters.

In [103]:
bs = 64

A stateful model is easy to create (just add "stateful=True") but harder to train. We had to add batchnorm and use LSTM to get reasonable results. It is because we get activate using the weight matrix for each loop of the RNN. This yields hundred of  thousands weights in the state matrix. The training was solved in the LSTM model, where we use a neural net inside the loop, that decides how much of the state matrix to keep and how much to use at each activation.

When using stateful in keras, you have to also add 'batch_input_shape' to the first layer, and fix the batch size there.

This model, when shuffling is set to false, allows a build up of arbitrarily long dependencies.

In [108]:
model = Sequential([
    Embedding(vocab_size, n_fac, input_length = cs, batch_input_shape = (bs, 8)),
    BatchNormalization(),
    LSTM(n_hidden, return_sequences = True, stateful = True),
    TimeDistributed(Dense(vocab_size, activation = 'softmax')),
    ])

We then compile the model.

In [109]:
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = Adam())

Since we're using a fixed batch shape, we have to ensure our inputs and outputs are a even multiple of the batch size.

In [110]:
mx = len(x_rnn) // bs * bs

We then fit the model, remembering that Shuffle has to be set as False, as the ordering of the data now has to be ensured in order to estimate the long term dependencies.

In [111]:
model.fit(x_rnn[:mx], y_rnn[:mx], batch_size = bs, epochs = 4, shuffle = False)

Epoch 1/4
75072/75072 [==============================] - 49s - loss: 2.2510    
Epoch 2/4
75072/75072 [==============================] - 49s - loss: 1.9769    
Epoch 3/4
75072/75072 [==============================] - 48s - loss: 1.8967    
Epoch 4/4
75072/75072 [==============================] - 49s - loss: 1.8497    


We lower the learning rate and run for some more epochs.

In [113]:
model.optimizer.lr = 1e-4

model.fit(x_rnn[:mx], y_rnn[:mx], batch_size = bs, epochs = 8, shuffle = False)

Epoch 1/8
75072/75072 [==============================] - 50s - loss: 1.8056    
Epoch 2/8
75072/75072 [==============================] - 50s - loss: 1.7832    
Epoch 3/8
75072/75072 [==============================] - 50s - loss: 1.7621    
Epoch 4/8
75072/75072 [==============================] - 50s - loss: 1.7436    
Epoch 5/8
75072/75072 [==============================] - 50s - loss: 1.7268    
Epoch 6/8
75072/75072 [==============================] - 50s - loss: 1.7115    
Epoch 7/8
75072/75072 [==============================] - 50s - loss: 1.6975    
Epoch 8/8
75072/75072 [==============================] - 50s - loss: 1.6844    


So using the state keeping model makes us able to lower the loss quite a bit, compared to the model not keeping the state.

We can do even better by adding a second RNN which uses the first RNN as input. See this in rnn-example.ipynb notebook.

## One-hot sequence model with keras

We have previously not one hot encoded out output. This is due tio the fact that Keras impelements the sparse categorical crossentropy loss function, which is identical to categorical crossentropy, but takes an integer target instead of a binary one hot encoded target. This is usefull, when you have very large vocabularies with hundreds of thousands words.

However, in order to make thins simpler, we will use one hot encoding, when creating our hand craftet Theano model. This is the keras version of the theano model that we're about to create.

So below we use normal cross enthropy and use no embedding layer.

In [115]:
model = Sequential([
        SimpleRNN(n_hidden, return_sequences = True, input_shape = (cs, vocab_size),
                  activation = 'relu', recurrent_initializer = 'identity'),
        TimeDistributed(Dense(vocab_size, activation = 'softmax')),
    ])

model.compile(loss = 'categorical_crossentropy', optimizer = Adam())

As we do not use an embedding layer and we use normal categorical crossenthropy, we have to one hot encode out input and labels.

In [116]:
oh_ys = [to_categorical(o, vocab_size) for o in ys]
oh_y_rnn=np.stack(oh_ys, axis=1)

oh_xs = [to_categorical(o, vocab_size) for o in xs]
oh_x_rnn=np.stack(oh_xs, axis=1)

oh_x_rnn.shape, oh_y_rnn.shape

((75110, 8, 86), (75110, 8, 86))

We can then fit this model, an see that it performs on the same level as the model using embeddings and sparse categorical crossentropy

In [117]:
model.fit(oh_x_rnn, oh_y_rnn, batch_size = 64, epochs = 8)

Epoch 1/8
75110/75110 [==============================] - 11s - loss: 2.4478    
Epoch 2/8
75110/75110 [==============================] - 11s - loss: 2.0380    
Epoch 3/8
75110/75110 [==============================] - 11s - loss: 1.9240    
Epoch 4/8
75110/75110 [==============================] - 11s - loss: 1.8594    
Epoch 5/8
75110/75110 [==============================] - 11s - loss: 1.8168    
Epoch 6/8
75110/75110 [==============================] - 11s - loss: 1.7862    
Epoch 7/8
75110/75110 [==============================] - 11s - loss: 1.7624    
Epoch 8/8
75110/75110 [==============================] - 11s - loss: 1.7436    


In [118]:
def get_nexts_oh(inp):
    idxs = np.array([char_indices[c] for c in inp])
    arr = to_categorical(idxs, vocab_size)
    p = model.predict(arr[np.newaxis,:])[0]
    print(list(inp))
    return [chars[np.argmax(o)] for o in p]

In [119]:
get_nexts_oh(' this is')

[' ', 't', 'h', 'i', 's', ' ', 'i', 's']


['t', 'h', 'e', 's', ' ', 'c', 's', ' ']

In [120]:
get_nexts_oh('part of ')

['p', 'a', 'r', 't', ' ', 'o', 'f', ' ']


['e', 'r', 't', ' ', 'o', 'f', ' ', 't']

In [121]:
get_nexts_oh('queens a')

['q', 'u', 'e', 'e', 'n', 's', ' ', 'a']


['u', 'e', 'n', 's', 'c', ' ', 'o', 'n']

## Theano RNN

In this section we build an RNN in pure Theano, using one hot encoding.

In [122]:
n_input = vocab_size
n_output = vocab_size

Using raw theano, we have to create our weight matrices and bias vectors ourselves - here are the functions we'll use to do so (using glorot initialization).
The return values are wrapped in shared(), which is how we tell theano that it can manage this data (copying it to and from the GPU as necessary).

In [141]:
def init_wgts(rows, cols):
    # Initialise weights with random numbers of given scale.
    
    # Calculate the scale of the random numbers
    scale = math.sqrt(2 / rows)
    
    # Create the random numbers from normal distribution. We use shared as wrapper
    # to pass it to Theano for later use in the GPU calculations.
    return shared(normal(scale = scale, size = (rows, cols)).astype(np.float32))

def init_bias(rows):
    # Intialise bias by using a simple 0 vector. We use shared as wrapper
    # to pass it to Theano for later use in the GPU calculations.
    
    return shared(np.zeros(rows, dtype = np.float32))

We return the weights and biases together as a tuple. For the hidden weights, we'll use an identity initialization (as recommended by [Hinton](https://arxiv.org/abs/1504.00941).)

In [142]:
def wgts_and_bias(n_in, n_out):
    # Generate weights and bias for input and output layers
    
    # n_in rows and n_out columns
    return init_wgts(n_in, n_out), init_bias(n_out)

def id_and_bias(n):
    # Generate weights and bias for hidden layers layers
    
    # Use identity matrix as initialisation as proposed by Hinton.
    return shared(np.eye(n, dtype = np.float32)), init_bias(n)

Theano doesn't actually do any computations until we explicitly compile and evaluate the function (at which point it'll be turned into CUDA code and sent off to the GPU). So our job is to describe the computations that we'll want theano to do - the first step is to tell theano what inputs we'll be providing to our computation. So we construct some input and output tensors in the form of matrixes, vectors and scalars.

In [129]:
# Specify variables.
t_inp = T.matrix('inp')
t_outp = T.matrix('outp')
t_h0 = T.vector('h0')
lr = T.scalar('lr')

all_args = [t_h0, t_inp, t_outp, lr]

Now we're ready to create our intial weight matrices.

In [143]:
# Weights and bias to hidden layer
W_h = id_and_bias(n_hidden)

# Weights and bias to input
W_x = wgts_and_bias(n_input, n_hidden)

# Weights and bias to output
W_y = wgts_and_bias(n_hidden, n_output)

# Gather in a single list
w_all = list(chain.from_iterable([W_h, W_x, W_y]))

Theano handles looping (in this case looping trough our RNN) by using the [GPU scan](http://http.developer.nvidia.com/GPUGems3/gpugems3_ch39.html) operation. We have to tell theano what to do at each step through the scan - this is the function we'll use, which does a single forward pass for one character:

In [145]:
def step(x, h, W_h, b_h, W_x, b_x, W_y, b_y):
    
    # Calculate the hidden activations
    h = nnet.relu(T.dot(x, W_x) + b_x + T.dot(h, W_h) + b_h)
    
    # Calculate the output activations
    y = nnet.softmax(T.dot(h, W_y) + b_y)
    
    # Return both (the 'Flatten()' is to work around a theano bug)
    return h, T.flatten(y, 1)

Now we can provide everything necessary for the scan operation, so we can setup that up - we have to pass in the function to call at each step, the sequence to step through, the initial values of the outputs, and any other arguments to pass to the step function.

In [147]:
# Perform one function for every sequence
[v_h, v_y], _ = theano.scan(step,
                            sequences = t_inp, 
                            outputs_info = [t_h0, None],
                            non_sequences = w_all)

We can now calculate our loss function, and all of our gradients, with just a couple of lines of code!

In [148]:
error = nnet.categorical_crossentropy(v_y, t_outp).sum()
g_all = T.grad(error, w_all)

We even have to show theano how to do SGD - so we set up this dictionary of updates to complete after every forward pass, which apply to standard SGD update rule to every weight.

In [150]:
def upd_dict(wgts, grads, lr): 
    return OrderedDict({w: w - g * lr for (w, g) in zip(wgts, grads)})

upd = upd_dict(w_all, g_all, lr)

We're finally ready to compile the function!

In [151]:
fn = theano.function(all_args, error, updates = upd, allow_input_downcast = True)

We can then specify the actual values for the variables.

In [152]:
X = oh_x_rnn
Y = oh_y_rnn
X.shape, Y.shape

((75110, 8, 86), (75110, 8, 86))

To use it, we simply loop through our input data, calling the function compiled above, and printing our progress from time to time.

In [153]:
err = 0.0
l_rate = 0.01

for i in range(len(X)): 
    err += fn(np.zeros(n_hidden), X[i], Y[i], l_rate)
    if i % 1000 == 999: 
        print ("Error:{:.3f}".format(err / 1000))
        err = 0.0

Error:25.154
Error:21.470
Error:20.950
Error:19.943
Error:18.830
Error:19.258
Error:19.059
Error:18.418
Error:17.941
Error:18.256
Error:17.512
Error:17.630
Error:18.476
Error:17.351
Error:16.812
Error:17.733
Error:17.370
Error:17.227
Error:16.858
Error:16.707
Error:16.542
Error:16.401
Error:16.709
Error:16.189
Error:16.872
Error:16.627
Error:16.081
Error:16.308
Error:16.277
Error:16.445
Error:16.746
Error:16.442
Error:16.660
Error:16.325
Error:16.041
Error:16.699
Error:15.950
Error:16.410
Error:16.091
Error:16.316
Error:15.419
Error:15.724
Error:15.798
Error:16.016
Error:16.054
Error:15.876
Error:15.606
Error:16.131
Error:16.002
Error:16.082
Error:15.262
Error:15.715
Error:14.957
Error:14.887
Error:15.621
Error:15.383
Error:14.756
Error:15.494
Error:15.201
Error:15.062
Error:15.061
Error:15.488
Error:15.378
Error:15.089
Error:14.783
Error:14.848
Error:14.372
Error:14.755
Error:15.298
Error:14.837
Error:15.139
Error:14.768
Error:14.447
Error:14.551
Error:14.558


We can then create a function calculate the vector of outputs.

In [154]:
f_y = theano.function([t_h0, t_inp], v_y, allow_input_downcast = True)

And then calculated the actual predictions.

In [155]:
pred = np.argmax(f_y(np.zeros(n_hidden), X[6]), axis=1)

In [156]:
act = np.argmax(X[6], axis = 1)

In [157]:
[indices_char[o] for o in act]

['t', 'h', 'e', 'n', '?', ' ', 'I', 's']

In [158]:
[indices_char[o] for o in pred]

['h', 'e', ' ', ' ', ' ', 'T', 't', ' ']

## Set up basic functions

Now we're going to try to repeat the above theano RNN, using just pure python (and numpy). Which means, we have to do everything ourselves, including defining the basic functions of a neural net! Below are all of the definitions, along with tests to check that they give the same answers as theano. The functions ending in _d are the derivatives of each function.

In [ ]:
def sigmoid(x): return 1/(1+np.exp(-x))
def sigmoid_d(x): 
    output = sigmoid(x)
    return output*(1-output)

In [ ]:
def relu(x): return np.maximum(0., x)
def relu_d(x): return (x > 0.)*1.

In [ ]:
relu(np.array([3.,-3.])), relu_d(np.array([3.,-3.]))

In [ ]:
relu(np.array([3.,-3.])), relu_d(np.array([3.,-3.]))

In [ ]:
def dist(a,b): return pow(a-b,2)
def dist_d(a,b): return 2*(a-b)

In [ ]:
eps = 1e-7
def x_entropy(pred, actual): 
    return -np.sum(actual * np.log(np.clip(pred, eps, 1-eps)))
def x_entropy_d(pred, actual): return -actual/pred

In [ ]:
def softmax(x): return np.exp(x)/np.exp(x).sum()

In [ ]:
def softmax_d(x):
    sm = softmax(x)
    res = np.expand_dims(-sm,-1)*sm
    res[np.diag_indices_from(res)] = sm*(1-sm)
    return res

In [ ]:
test_preds = np.array([0.2,0.7,0.1])
test_actuals = np.array([0.,1.,0.])
nnet.categorical_crossentropy(test_preds, test_actuals).eval()

In [ ]:
x_entropy(test_preds, test_actuals)

In [ ]:
test_inp = T.dvector()
test_out = nnet.categorical_crossentropy(test_inp, test_actuals)
test_grad = theano.function([test_inp], T.grad(test_out, test_inp))

In [ ]:
test_grad(test_preds)

In [ ]:
x_entropy_d(test_preds, test_actuals)

In [ ]:
pre_pred = random(oh_x_rnn[0][0].shape)
preds = softmax(pre_pred)
actual = oh_x_rnn[0][0]

In [ ]:
np.allclose(softmax_d(pre_pred).dot(loss_d(preds,actual)), preds-actual)

In [ ]:
softmax(test_preds)

In [ ]:
nnet.softmax(test_preds).eval()

In [ ]:
test_out = T.flatten(nnet.softmax(test_inp))

In [ ]:
test_grad = theano.function([test_inp], theano.gradient.jacobian(test_out, test_inp))

In [ ]:
test_grad(test_preds)

In [ ]:
softmax_d(test_preds)

In [ ]:
act=relu
act_d = relu_d

In [ ]:
loss=x_entropy
loss_d=x_entropy_d

In [ ]:
def scan(fn, start, seq):
    res = []
    prev = start
    for s in seq:
        app = fn(prev, s)
        res.append(app)
        prev = app
    return res

In [ ]:
scan(lambda prev,curr: prev+curr, 0, range(5))

### Set up training

Let's now build the functions to do the forward and backward passes of our RNN. First, define our data and shape.

In [ ]:
inp = oh_x_rnn
outp = oh_y_rnn
n_input = vocab_size
n_output = vocab_size

In [ ]:
inp.shape, outp.shape

Here's the function to do a single forward pass of an RNN, for a single character.

In [ ]:
def one_char(prev, item):
    # Previous state
    tot_loss, pre_hidden, pre_pred, hidden, ypred = prev
    # Current inputs and output
    x, y = item
    pre_hidden = np.dot(x,w_x) + np.dot(hidden,w_h)
    hidden = act(pre_hidden)
    pre_pred = np.dot(hidden,w_y)
    ypred = softmax(pre_pred)
    return (
        # Keep track of loss so we can report it
        tot_loss+loss(ypred, y),
        # Used in backprop
        pre_hidden, pre_pred, 
        # Used in next iteration
        hidden, 
        # To provide predictions
        ypred)

We use scan to apply the above to a whole sequence of characters.

In [ ]:
def get_chars(n): return zip(inp[n], outp[n])
def one_fwd(n): return scan(one_char, (0,0,0,np.zeros(n_hidden),0), get_chars(n))

Now we can define the backward step. We use a loop to go through every element of the sequence. The derivatives are applying the chain rule to each step, and accumulating the gradients across the sequence.

In [ ]:
# "Columnify" a vector
def col(x): return x[:,newaxis]

def one_bkwd(args, n):
    global w_x,w_y,w_h

    i=inp[n]  # 8x86
    o=outp[n] # 8x86
    d_pre_hidden = np.zeros(n_hidden) # 256
    for p in reversed(range(len(i))):
        totloss, pre_hidden, pre_pred, hidden, ypred = args[p]
        x=i[p] # 86
        y=o[p] # 86
        d_pre_pred = softmax_d(pre_pred).dot(loss_d(ypred,y))  # 86
        d_pre_hidden = (np.dot(d_pre_hidden, w_h.T) 
                        + np.dot(d_pre_pred,w_y.T)) * act_d(pre_hidden) # 256

        # d(loss)/d(w_y) = d(loss)/d(pre_pred) * d(pre_pred)/d(w_y)
        w_y -= col(hidden) * d_pre_pred * alpha
        # d(loss)/d(w_h) = d(loss)/d(pre_hidden[p-1]) * d(pre_hidden[p-1])/d(w_h)
        if (p>0): w_h -= args[p-1][3].dot(d_pre_hidden) * alpha
        w_x -= col(x)*d_pre_hidden * alpha
    return d_pre_hidden

Now we can set up our initial weight matrices. Note that we're not using bias at all in this example, in order to keep things simpler.

In [ ]:
scale=math.sqrt(2./n_input)
w_x = normal(scale=scale, size=(n_input,n_hidden))
w_y = normal(scale=scale, size=(n_hidden, n_output))
w_h = np.eye(n_hidden, dtype=np.float32)

Our loop looks much like the theano loop in the previous section, except that we have to call the backwards step ourselves.

In [ ]:
overallError=0
alpha=0.0001
for n in range(10000):
    res = one_fwd(n)
    overallError+=res[-1][0]
    deriv = one_bkwd(res, n)
    if(n % 1000 == 999):
        print ("Error:{:.4f}; Gradient:{:.5f}".format(
                overallError/1000, np.linalg.norm(deriv)))
        overallError=0

## Keras GRU

Identical to the last keras rnn, but a GRU!

In [ ]:
model=Sequential([
        GRU(n_hidden, return_sequences=True, input_shape=(cs, vocab_size),
                  activation='relu', inner_init='identity'),
        TimeDistributed(Dense(vocab_size, activation='softmax')),
    ])
model.compile(loss='categorical_crossentropy', optimizer=Adam())

In [ ]:
model.fit(oh_x_rnn, oh_y_rnn, batch_size=64, nb_epoch=8)

In [ ]:
get_nexts_oh(' this is')

## Theanu GRR

### Separate weights

The theano GRU looks just like the simple theano RNN, except for the use of the reset and update gates. Each of these gates requires its own hidden and input weights, so we add those to our weight matrices.

In [ ]:
W_h = id_and_bias(n_hidden)
W_x = init_wgts(n_input, n_hidden)
W_y = wgts_and_bias(n_hidden, n_output)
rW_h = init_wgts(n_hidden, n_hidden)
rW_x = wgts_and_bias(n_input, n_hidden)
uW_h = init_wgts(n_hidden, n_hidden)
uW_x = wgts_and_bias(n_input, n_hidden)
w_all = list(chain.from_iterable([W_h, W_y, uW_x, rW_x]))
w_all.extend([W_x, uW_h, rW_h])

Here's the definition of a gate - it's just a sigmoid applied to the addition of the dot products of the input vectors.

In [ ]:
def gate(x, h, W_h, W_x, b_x):
    return nnet.sigmoid(T.dot(x, W_x) + b_x + T.dot(h, W_h))

Our step is nearly identical to before, except that we multiply our hidden state by our reset gate, and we update our hidden state based on the update gate.

In [ ]:
def step(x, h, W_h, b_h, W_y, b_y, uW_x, ub_x, rW_x, rb_x, W_x, uW_h, rW_h):
    reset = gate(x, h, rW_h, rW_x, rb_x)
    update = gate(x, h, uW_h, uW_x, ub_x)
    h_new = gate(x, h * reset, W_h, W_x, b_h)
    h = update*h + (1-update)*h_new
    y = nnet.softmax(T.dot(h, W_y) + b_y)
    return h, T.flatten(y, 1)

Everything from here on is identical to our simple RNN in theano.

In [ ]:
[v_h, v_y], _ = theano.scan(step, sequences=t_inp, 
                            outputs_info=[t_h0, None], non_sequences=w_all)

In [ ]:
error = nnet.categorical_crossentropy(v_y, t_outp).sum()
g_all = T.grad(error, w_all)

In [ ]:
upd = upd_dict(w_all, g_all, lr)
fn = theano.function(all_args, error, updates=upd, allow_input_downcast=True)

In [ ]:
err=0.0; l_rate=0.1
for i in range(len(X)): 
    err+=fn(np.zeros(n_hidden), X[i], Y[i], l_rate)
    if i % 1000 == 999: 
        l_rate *= 0.95
        print ("Error:{:.2f}".format(err/1000))
        err=0.0

### Combined weights

We can make the previous section simpler and faster by concatenating the hidden and input matrices and inputs together. We're not going to step through this cell by cell - you'll see it's identical to the previous section except for this concatenation.

In [ ]:
W = (shared(np.concatenate([np.eye(n_hidden), normal(size=(n_input, n_hidden))])
            .astype(np.float32)), init_bias(n_hidden))

rW = wgts_and_bias(n_input+n_hidden, n_hidden)
uW = wgts_and_bias(n_input+n_hidden, n_hidden)
W_y = wgts_and_bias(n_hidden, n_output)
w_all = list(chain.from_iterable([W, W_y, uW, rW]))

In [ ]:
def gate(m, W, b): return nnet.sigmoid(T.dot(m, W) + b)

In [ ]:
def step(x, h, W, b, W_y, b_y, uW, ub, rW, rb):
    m = T.concatenate([h, x])
    reset = gate(m, rW, rb)
    update = gate(m, uW, ub)
    m = T.concatenate([h*reset, x])
    h_new = gate(m, W, b)
    h = update*h + (1-update)*h_new
    y = nnet.softmax(T.dot(h, W_y) + b_y)
    return h, T.flatten(y, 1)

In [ ]:
[v_h, v_y], _ = theano.scan(step, sequences=t_inp, 
                            outputs_info=[t_h0, None], non_sequences=w_all)

In [ ]:
def upd_dict(wgts, grads, lr): 
    return OrderedDict({w: w-g*lr for (w,g) in zip(wgts,grads)})

In [ ]:
error = nnet.categorical_crossentropy(v_y, t_outp).sum()
g_all = T.grad(error, w_all)

In [ ]:
upd = upd_dict(w_all, g_all, lr)
fn = theano.function(all_args, error, updates=upd, allow_input_downcast=True)

In [ ]:
err=0.0; l_rate=0.01
for i in range(len(X)): 
    err+=fn(np.zeros(n_hidden), X[i], Y[i], l_rate)
    if i % 1000 == 999: 
        print ("Error:{:.2f}".format(err/1000))
        err=0.0